# Climate Analysis and Exploration

In [1]:
# Load Modules

%matplotlib inline
# Customizable style sheets and parameters for matplotlib
from matplotlib import style
style.use('fivethirtyeight')
# Plot generator for bar, pie, line, scatter, box and other plots
import matplotlib.pyplot as plt
# High-level math and basic algebra computing tool
import numpy as np
# Dataframe generator that prints out various types of datasets
import pandas as pd
# Date and time attributes such as year, month, day, hour, minute, second 
import datetime as dt

In [2]:
# Load Modules

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
# File Path

# Define variable that represents the file path location
sqlite_file_path = "Resources/hawaii.sqlite"

In [4]:
# Create Engine

# Define variable that holds the create engine function to read sqlite file
sqlite_engine = create_engine(f"sqlite:///{sqlite_file_path}")

## Reflect Tables into SQLAlchemy ORM

In [5]:
# Reflect Tables

# Define the automap variable to be used in the table reflect
base = automap_base()
# Reflect the tables from within the .sqlite file
base.prepare(sqlite_engine, reflect=True)

In [6]:
# Detect Classes

# View all of the classes that automap found
base.classes.keys()

['measurement', 'station']

In [7]:
# Define Classes

# Define variable that references each found class name
Measurement = base.classes.measurement
Station = base.classes.station

In [8]:
# Identify Columns

# Define variable for inspect function to get column names from dataset
inspector = inspect(sqlite_engine)
inspector.get_table_names()

# Define variable that represents the get columns function from the measurement class
column_names = inspector.get_columns('Measurement')
for column in column_names:
    print(column['name'], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [9]:
sqlite_engine.execute('SELECT * FROM Measurement LIMIT 1').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0)]

In [10]:
# Start Session

# Create our session (link) from Python to the DB
session = Session(sqlite_engine)

# Exploratory Precipitation Analysis

In [14]:
# Specific Query

# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

In [16]:
# Specific Query

# Define variable that finds the date one year ago to the day from the most recent date in the dataset
last_year = dt.date(2017,8,23) - dt.timedelta(days=365)
print(last_year)

2016-08-23


In [27]:
# Specific Query

# Define variable that represents the query of the last 12 months of Percipitation data
last_years_percipitation_data = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= last_year).\
    order_by(Measurement.date).all()
# print(last_years_percipitation_data)

In [30]:
# Specific Query

# Define variable that represents a query to retrieve all the data and precipitation scores
total_scores = session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.desc()).all()
# print(total_scores)

In [31]:
# Pandas Database

# Save the query results as a Pandas DataFrame and set the index to the date column
percipitation_dataframe = pd.DataFrame(last_years_percipitation_data, columns=["Date","Precipitation"])
percipitation_dataframe.set_index("Date", inplace=True,)
percipitation_dataframe.head()

,Precipitation
Date,
2016-08-23,0.00
2016-08-23,0.15
2016-08-23,0.05
2016-08-23,NaN
2016-08-23,0.02


In [ ]:







# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()